<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/ENEM4%20bit%20Open-LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U rank_bm25
!pip install -q -U datasets
!pip install -q -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
# ---------------------------------------------------------- #
# This code is from https://github.com/piresramon/gpt-4-enem #
# ---------------------------------------------------------- #

import json
import torch
import random 
import collections

import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 400)

from tqdm.auto import tqdm

import rank_bm25

from transformers import LlamaTokenizer, LlamaForCausalLM
from sklearn.metrics import accuracy_score
from datasets import Dataset

if torch.cuda.is_available(): 
    device = 'cuda'  
else: 
    device ='cpu'

MANUAL_SEED = 2711
rnd = random.Random()
rnd.seed(MANUAL_SEED)
def deterministic(rep=True, manual_seed=MANUAL_SEED):
    if rep:
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Experimento deterministico, seed: {manual_seed}')
        if device == 'cuda':
            print(f'Existe {torch.cuda.device_count()} GPU\
            {torch.cuda.get_device_name(0)} disponível.')
    else:
        print('Experimento randomico')
deterministic()    

Experimento deterministico, seed: 2711
Existe 1 GPU            NVIDIA A100-SXM4-40GB disponível.


# Get Enem json and create enem_data 

In [2]:
def ignore_question(doc):
    filters = {
        'IU': False,
        # 'MR': False,  # uncomment to filter out MR
        # 'CE': False,  # uncomment to filter out CE
        'ML': False,
    }
    for k,v in filters.items():
        if doc[k] != v:
            return True
    return False

def _process_doc_cot(doc):
    def format_example(doc, choices):
        prompt = "Contexto: " + doc["context"] + "\n"
        prompt += "Enunciado: " + doc["question"] + "\nAlternativas:\n"
        for choice, option in zip(choices, doc["options"]):
            prompt += f"{choice.upper()}. {option}\n"
        
        prompt += "Explicação: " + doc.get("explanation", "")
        return prompt.strip()
    choices = ['a', 'b', 'c', 'd', 'e']
    return {
        "query": format_example(doc, choices),
        "choices": doc["options"],
        "gold": choices.index(doc["label"]),
        "id": doc["id"],
        "exam": doc["exam"],
    }    

dataset = collections.defaultdict(list)
        
data_path = "/content/drive/MyDrive/LLMs/ENEM/ENEMdataset/2022.json"
with open(data_path) as f:
    documents = json.load(f)

documents = list(filter(lambda doc: not ignore_question(doc), documents))
dataset['test'] = list(map(_process_doc_cot, documents))

enem_data = {ix:doc  for ix, doc in enumerate(dataset['test'])}

print(f"tamanho dataset: {len(enem_data)}")

enem_data[0]    

tamanho dataset: 118


{'query': 'Contexto: A conquista da medalha de prata por Rayssa Leal, no skate street nos Jogos Olímpicos, é exemplo da representatividade feminina no esporte, avalia a âncora do jornal da rede de televisão da CNN. A apresentadora, que também anda de skate, celebrou a vitória da brasileira, que entrou para a história como a atleta mais nova a subir num pódio defendendo o Brasil. “Essa representatividade do esporte nos Jogos faz pensarmos que não temos que ficar nos encaixando em nenhum lugar. Posso gostar de passar notícia e, mesmo assim, gostar de skate, subir montanha, mergulhar, andar de bike, fazer yoga. Temos que parar de ficar enquadrando as pessoas dentro de regras. A gente vive num padrão no qual a menina ganha boneca, mas por que também não fazer um esporte de aventura? Por que o homem pode se machucar, cair de joelhos, e a menina tem que estar sempre lindinha dentro de um padrão? Acabamos limitando os talentos das pessoas”, afirmou a jornalista, sobre a prática do skate por m

# Create Prompts
- #### Dynamic Prompt (fewshot sampled)
- #### Dynamic Similar Prompt (bm25 rank fewshot)

In [3]:
def dynamic_similar_prompt(doc_id, data, topk=3):
    key_predict = [k for k, v in enem_data.items() if v['id'].startswith(doc_id)][0]
    query_list = [v['query'] for k,v in data.items()]
    tokenized_corpus = [doc.split(" ") for doc in query_list]
    bm25 = rank_bm25.BM25Plus(tokenized_corpus)

    query = data[key_predict]['query']
    tokenized_query = query.split(" ")
    
    doc_scores = bm25.get_scores(tokenized_query)
    
    # [1:] remove query with itself ---> topk+1
    top_similar_idxs = list(doc_scores.argsort()[-(topk+1):][::-1])[1:]
    if key_predict in top_similar_idxs: # k in topk_sim ---> must be false
        print('Error BM25 prompt similar retrieval')
    
    return top_similar_idxs


def dynamyc_fewshot_examples(num_fewshot, doc_id, sim_prompt=True, data=enem_data):
    if sim_prompt:
        topk_sim = dynamic_similar_prompt(doc_id, data, topk=num_fewshot)
        fewshot_ex = [enem_data[k] for k in topk_sim]

    else:
        # filter the sample in current predict from data
        all_possible_fewshot_keys = [k for k, v in data.items() if not v['id'].startswith(doc_id)]
        fewshot_keys = rnd.sample(all_possible_fewshot_keys, num_fewshot)
        fewshot_ex = [enem_data[k] for k in fewshot_keys]

    return fewshot_ex


def fewshot_context(doc, num_fewshot, similar_prompt=True, enem_data=enem_data):
    if num_fewshot == 0 and fewshotex is None:
        labeled_examples = ""
    else:
        fewshotex = dynamyc_fewshot_examples(
            num_fewshot=num_fewshot, doc_id=doc['id'], sim_prompt=similar_prompt, data=enem_data
        )
        
        labeled_examples = "Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional" + "\n"
        for i, doc_ex in enumerate(fewshotex):
            labeled_examples += f'Questão {i+1}:\n'
            labeled_examples += doc_ex['query'] + " " + ['A.', 'B.', 'C.', 'D.', 'E.'][doc_ex['gold']].upper()
            labeled_examples += '\n##\n'
        labeled_examples += f'Questão {len(fewshotex) + 1}:\n'

    example = doc['query']
    
    return labeled_examples + example

# -------------------------------------------------------------------------------------- #
# number of fewshots, must be > 0

# create inputs ---> list with fewshot samples + question to be answer 
one_shot, two_shot, three_shot = [],[],[]
for k,doc in enem_data.items():
    one_shot.append(fewshot_context(doc=doc,   num_fewshot=1, similar_prompt=False, enem_data=enem_data))
    two_shot.append(fewshot_context(doc=doc,   num_fewshot=2, similar_prompt=False, enem_data=enem_data))
    three_shot.append(fewshot_context(doc=doc, num_fewshot=3, similar_prompt=False, enem_data=enem_data))

In [4]:
one_shot[0]

'Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional\nQuestão 1:\nContexto: Macrocefalia urbana pode ser entendida como a massiva concentração das atividades econômicas em algumas metrópoles que propicia o desencadeamento de processos descompassados: redirecionamento e convergência de fluxos migratórios, déficit no número de empregos, ocupação desordenada de determinadas regiões da cidade e estigmatização de estratos sociais, que comprometem substancialmente a segurança pública urbana. SANTOS, M. O espaço dividido: os dois circuitos da economia urbana dos países subdesenvolvidos. São Paulo: Edusp, 2004.\nEnunciado: O processo de concentração espacial apresentado foi estimulado por qual fator geográfico?\nAlternativas:\nA. Limitação da área ocupada.\nB. Êxodo da população do campo.\nC. Ampliação do risco habitacional. D\nD. Deficiência do transporte alternativo.\nE. Crescimento da taxa de fecundidade.\nExplicação: B.\n##\nQuestão 2:\nContexto:

In [5]:
def get_results(trues, preds, chatgpt=False):
    dataframe = pd.DataFrame({'true': trues, 'pred':preds})

    dataframe['pred'] = dataframe.pred.apply(lambda x: x[-1:])
    
    map_label = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E"}
    dataframe['true'] = dataframe.true.apply(lambda x: map_label[x])
    
    acc = accuracy_score(dataframe['true'], dataframe['pred'])
    
    return dataframe, acc

In [19]:
model_path ="open_llama_7b_preview_700bt/open_llama_7b_preview_700bt_transformers_weights"


In [ ]:
AutoTokenizer.from_pretrained(model_path)


In [11]:
model_ckpt = "openlm-research/open_llama_7b_700bt_preview"
tokenizer = LlamaTokenizer.from_pretrained(model_ckpt)

def get_input_lenghts(dataset, tokenizer):
    num_words = [len(x.split()) for x in dataset]
    num_words = torch.tensor(num_words, dtype=torch.float)
    tokens_lengths = [len(tokenizer.encode(seq)) for seq in dataset]
    tokens_lengths = torch.tensor(tokens_lengths, dtype=torch.float)
    
    return int(torch.ceil(num_words.mean()).item()), tokens_lengths

# -------------------------------------------------------------------- #
num_words, ollama_lengths = get_input_lenghts(three_shot, tokenizer)
min_ollama = torch.ceil(ollama_lengths.min()).item()
max_ollama = torch.ceil(ollama_lengths.max()).item()
mean_ollama = torch.ceil(ollama_lengths.mean()).item()
# -------------------------------------------------------------------- #
print('\n\n')
print('--'*33)
print(f' Prompt 3-shot com média de {num_words} palavras em  ENEM2022 dataset')
print('--'*33)
print(f'O-LLaMA tem média de {int(mean_ollama)} tokens em 3-shot')




------------------------------------------------------------------
 Prompt 3-shot com média de 750 palavras em  ENEM2022 dataset
------------------------------------------------------------------
GPT2 tem média de 1830 tokens em 3-shot


# Eval 4-bit Open-LLaMA


In [14]:
model = LlamaForCausalLM.from_pretrained(model_ckpt, load_in_4bit=True, device_map="auto")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
deterministic()    

trues, preds = [], []
loop = tqdm(one_shot, leave=True)

for ix, batch in enumerate(loop):

    # to make sure the same example to predict has the correct label 
    # find the point where the question to be evaluate starts
    point = one_shot[ix].find('Questão 2:\n')
    
    # "11+point" ---> is the start of the query-text, so this is a naive approach
    if enem_data[ix]['query'] == one_shot[ix][11+point:]:
        inputs = tokenizer.encode(one_shot[ix], return_tensors="pt", max_length=2048).to(device)
        outputs = model.generate(inputs)
        
        preds.append(tokenizer.decode(outputs[0]))
        trues.append(enem_data[ix]['gold'])
    else:
        # if the query-text is different from inputs to be evaluated 
        # we print for further investigation
        print(point, ix)

Experimento deterministico, seed: 2711
Existe 1 GPU            NVIDIA A100-SXM4-40GB disponível.


  0%|          | 0/118 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1349: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 915, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 1101, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider

In [16]:
preds[0]

' ⁇  Dado o exemplo, forneça uma resposta após ##Explicação:## não faça nenhuma explicação adicional\nQuestão 1:\nContexto: Macrocefalia urbana pode ser entendida como a massiva concentração das atividades econômicas em algumas metrópoles que propicia o desencadeamento de processos descompassados: redirecionamento e convergência de fluxos migratórios, déficit no número de empregos, ocupação desordenada de determinadas regiões da cidade e estigmatização de estratos sociais, que comprometem substancialmente a segurança pública urbana. SANTOS, M. O espaço dividido: os dois circuitos da economia urbana dos países subdesenvolvidos. São Paulo: Edusp, 2004.\nEnunciado: O processo de concentração espacial apresentado foi estimulado por qual fator geográfico?\nAlternativas:\nA. Limitação da área ocupada.\nB. Êxodo da população do campo.\nC. Ampliação do risco habitacional. D\nD. Deficiência do transporte alternativo.\nE. Crescimento da taxa de fecundidade.\nExplicação: B.\n##\nQuestão 2:\nConte

In [17]:
dataframe = pd.DataFrame({'true': trues, 'pred':preds})
dataframe.pred.apply(lambda x: x[-1:])

0      C
1      C
2      C
3      C
4      C
5      E
6      E
7      C
8      C
9      C
10     D
11     E
12     C
13     C
14     C
15     C
16     A
17     C
18     C
19     E
20     B
21     C
22     C
23     E
24     C
25     C
26     A
27     E
28     C
29     C
30     A
31     D
32     E
33     C
34     C
35     E
36     A
37     C
38     E
39     B
40     C
41     B
42     C
43     E
44     E
45     E
46     E
47     A
48     E
49     A
50     E
51     D
52     C
53     A
54     A
55     C
56     C
57     C
58     A
59     C
60     C
61     C
62     A
63     D
64     C
65     C
66     A
67     E
68     C
69     A
70     E
71     C
72     D
73     D
74     C
75     A
76     E
77     C
78     A
79     A
80     C
81     C
82     B
83     C
84     C
85     A
86     C
87     C
88     C
89     C
90     C
91     C
92     A
93     C
94     A
95     E
96     A
97     E
98     A
99     E
100    C
101    A
102    E
103    C
104    E
105    C
106    C
107    A
108    A
109    C
110    C
1

In [18]:
gptj, acc = get_results(trues, preds)

print(f'ACC of 1-shot GPT0J on ENEM-2022: {acc:.3}')

ACC of 1-shot GPT0J on ENEM-2022: 0.22
